In [23]:
%pip install brotli

  Using cached Brotli-1.1.0-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
Using cached Brotli-1.1.0-cp312-cp312-win_amd64.whl (357 kB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
import requests
import json
import brotli

url = "https://jlcpcb.com/api/overseas-pcb-order/v1/shoppingCart/smtGood/selectSmtComponentList/v2"

headers = {
    "Host": "jlcpcb.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-GB,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Content-Type": "application/json",
    "secretkey": "  YOUR SECRETKEY GOES HERE :)",#
    "X-XSRF-TOKEN": "  YOUR TOKEN GOES HERE :)",#
    "Origin": "https://jlcpcb.com",
    "DNT": "1",
    "Connection": "keep-alive",
    "Referer": "https://jlcpcb.com/parts/basic_parts",
    "Cookie": "  YOUR COOKIE GOES HERE :)",#
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Priority": "u=0",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
}

for page in range(33, 48):
    data = {
        "currentPage": page,
        "pageSize": 100,
        "keyword": None,
        "componentLibraryType": "base",
        "preferredComponentFlag": True,
        "stockFlag": None,
        "stockSort": None,
        "firstSortName": None,
        "secondSortName": None,
        "componentBrand": None,
        "componentSpecification": None,
        "componentAttributes": [],
        "searchSource": "search",
    }

    response = requests.post(url, headers=headers, json=data)

    print(f"Page {page} status code: {response.status_code}")
    print(f"Page {page} headers: {response.headers}")

    if response.status_code == 200:
        filename = f"scraped/response{page}.json"
        if (
            "Content-Encoding" in response.headers
            and response.headers["Content-Encoding"] == "br"
        ):
            with open(filename, "wb") as f:
                f.write(brotli.decompress(response.content))
        else:
            with open(filename, "w") as f:
                f.write(response.text)
        print(f"Response written to {filename}")
    else:
        print(f"Failed to retrieve data for page {page}")

Page 33 status code: 200
Page 33 headers: {'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'nginx/1.22.0', 'Date': 'Tue, 10 Sep 2024 01:45:23 GMT', 'J-Trace-UA': 'ua:jQrM5Q78lW9CPN', 'J-Trace-ID': 'bd7af963d62148718453e3c341f2b3d5', 'Set-Cookie': 'XSRF-TOKEN=79975e53-425f-40de-8900-3ca1bee09c96; Max-Age=1800; Expires=Tue, 10 Sep 2024 02:15:23 GMT; Path=/, jlc_s=A; Max-Age=150196224; Expires=Thu, 14 Jun 2029 10:55:47 GMT; Domain=jlcpcb.com; Path=/, JSESSIONID=FB9A4EA2A306EB680D620088E40796F8; Path=/; Secure; HttpOnly', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache,no-store,must-revalidate', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 272dcd56e8473f3d7bddd621c56c361a.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'AKL50-C1', 'X-Amz-Cf-Id': 'hQi6X0CDoITjTT2X4WjbLleccrEhtdC8pykauI19mm6NjE3ABqt3mg==', 'x-xsrf-token': ''}
Response written to response33.json
Page 34 status 

In [34]:
import json
import csv

component_codes = []

for i in range(1, 48):
    filename = f"scraped/response{i}.json"
    with open(filename, "r") as f:
        data = json.load(f)
        for component in (
            data.get("data", {}).get("componentPageInfo", {}).get("list", [])
        ):
            component_code = component.get("componentCode")
            if component_code and component_code.startswith("C"):
                component_codes.append(component_code[1:])  # Remove the "C" prefix

with open("scraped/ComponentList.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["lcsc"])  # header
    writer.writerows([[code] for code in component_codes])